In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
device = 'cuda:0' if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [2]:
dataset = torchvision.datasets.FashionMNIST(root=".", train=True, download=True, transform=transforms.ToTensor())
x_train = dataset.data
y_train = dataset.targets
print(x_train.shape)

torch.Size([60000, 28, 28])


In [3]:
print(set(y_train.tolist()))
import numpy as np
x_train = torch.from_numpy(np.expand_dims(x_train, -1))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [4]:
print(x_train.shape)

torch.Size([60000, 28, 28, 1])


In [7]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
    nn.ReLU(),
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2),
    nn.ReLU(),
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2),
    nn.ReLU(),
    nn.Flatten(),
    nn.Dropout(0.2),
    nn.Linear(128 * 2 * 2, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, 10),
)
model = model.to(device)
x_train = x_train.to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [12]:
batch_size = 100
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=batch_size,
                                     shuffle=True)

In [13]:
epochs = 10
train_losses = np.zeros(epochs)
for it in range(epochs):
    model.train()
    losses = []
    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        losses.append(loss.item())
    
    train_lo = np.mean(losses)
    train_losses[it] = train_lo
    print(f'Epoch {it+1}/{epochs} Loss: {train_lo:.4f}')

Epoch 1/10 Loss: 0.6268
Epoch 2/10 Loss: 0.4209
Epoch 3/10 Loss: 0.3653
Epoch 4/10 Loss: 0.3302
Epoch 5/10 Loss: 0.2998
Epoch 6/10 Loss: 0.2790
Epoch 7/10 Loss: 0.2579
Epoch 8/10 Loss: 0.2414
Epoch 9/10 Loss: 0.2268
Epoch 10/10 Loss: 0.2142


In [27]:
# (x_train[400]/255).unsqueeze(0).shape
test = (x_train[400]/255).permute(2, 0, 1).unsqueeze(0)
_, prediction = torch.max(model(test), 1)
print(prediction)
print(y_train[400])

tensor([2], device='cuda:0')
tensor(2)
